Definição do Problema:

O objetivo deste trabalho foi criar um modelo de Machine Learning para prever o gasto mensal com energia de diferentes endereços, utilizando dados de consumo e características relacionadas à produção de energia solar. Essa previsão é útil para otimizar a gestão de recursos energéticos, identificar padrões de consumo e fornecer informações para iniciativas de economia.


In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import json



In [7]:
# Função para carregar arquivos JSON com a codificação correta
def carregar_json(caminho):
    with open(caminho, "r", encoding="ISO-8859-1") as arquivo:
        return pd.read_json(arquivo)

1. Carregar os arquivos JSON:
- Este trecho do código importa os arquivos usuarios.json, enderecos.json e energia_solar.json.
- Esses arquivos contêm os dados necessários para criar um modelo de regressão que prevê o gasto mensal de energia.
- As bibliotecas pandas e json são utilizadas para carregar e manipular os dados.


In [10]:
# Carregar os arquivos
usuarios = carregar_json("usuarios.json")
enderecos = carregar_json("enderecos.json")
energia_solar = carregar_json("energia_solar.json")

In [9]:
# Visualizar os dados
print(usuarios.head())
print(enderecos.head())
print(energia_solar.head())

   id_user      nome               email
0        1     Maria     maria@email.com
1        2      João      joao@email.com
2        3       Ana       ana@email.com
3        4    Carlos    carlos@email.com
4        5  Fernanda  fernanda@email.com
   id_endereco  fk_usuario tipo_residencial       cep  tarifa  gasto_mensal  \
0            1           2      Residencial  12345678    0.15           500   
1            2           2      Residencial  12345678    0.15           500   
2            3           4      Residencial  56789012    0.13           450   
3            4           6      Residencial  34567890    0.11           400   
4            5           7        Comercial  22334455    0.18          1000   

          nome  economia  
0  Apartamento       200  
1  Apartamento       200  
2      Chácara       350  
3      Sobrado       250  
4   Escritório       300  
   id_energia_solar  area_placa  irradiacao_solar  energ_est_gerada  \
0                 1           4               

2. Realizar o merge das tabelas:
- A tabela enderecos é combinada com energia_solar usando a chave estrangeira fk_endereco.
- O objetivo é criar um dataset consolidado que inclua informações de endereços e características de energia solar.
- A opção how='left' garante que todos os registros da tabela enderecos sejam mantidos, mesmo que não haja correspondência na tabela energia_solar.

In [12]:
# Merge dos dados
enderecos = enderecos.merge(energia_solar, left_on='id_endereco', right_on='fk_endereco', how='left')



3. Selecionar variáveis relevantes para o modelo:
- As variáveis selecionadas incluem:
  - **tipo_residencial**: Tipo do endereço (Residencial, Comercial, etc.).
  - **tarifa**: Custo por unidade de energia consumida.
  - **economia**: Valor economizado com energia solar.
  - **area_placa** e **irradiacao_solar**: Dados de energia solar.
  - **gasto_mensal**: A variável dependente (target), que queremos prever.

In [13]:
# Selecionar variáveis relevantes
dados = enderecos[[
    'tipo_residencial', 'tarifa', 'gasto_mensal', 'economia',
    'area_placa', 'irradiacao_solar', 'energ_est_gerada'
]]


4. Converter variáveis categóricas em numéricas:
- A variável tipo_residencial contém valores categóricos (e.g., "Residencial", "Comercial").
- Para treinar o modelo, essas categorias precisam ser convertidas em números usando .cat.codes.

In [14]:
# Converter variáveis categóricas em numéricas
dados['tipo_residencial'] = dados['tipo_residencial'].astype('category').cat.codes



<ipython-input-14-f82b426a20bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['tipo_residencial'] = dados['tipo_residencial'].astype('category').cat.codes


5. Remover linhas com valores ausentes:
- Linhas contendo valores **NaN** ou ausentes são excluídas para garantir que o modelo receba apenas dados completos.

In [15]:
# Remover linhas com valores ausentes
dados = dados.dropna()

6. Separar variáveis independentes (X) e a variável dependente (y):
- **X** contém as variáveis preditoras, que serão usadas para fazer previsões.
- **y** é a variável-alvo, ou seja, o gasto mensal que queremos prever.

In [16]:
# Separar variáveis independentes (X) e dependente (y)
X = dados.drop(columns=['gasto_mensal'])
y = dados['gasto_mensal']

7. Dividir os dados em conjuntos de treino e teste:
- Os dados são divididos em:
  - Conjunto de treino (80%): Para treinar o modelo.
  - Conjunto de teste (20%): Para avaliar o desempenho do modelo.
- A divisão é feita aleatoriamente usando train_test_split.

In [17]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

8. Treinar o modelo:
- Utilizamos o algoritmo **Random Forest Regressor**, um modelo de aprendizado baseado em múltiplas árvores de decisão.
- Ele é robusto, lida bem com dados tabulares e tem bom desempenho em problemas de regressão.

In [18]:
# Treinar o modelo
modelo = RandomForestRegressor(random_state=42)
modelo.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

9. Fazer previsões e avaliar o modelo:
- Após o treinamento, o modelo prevê o gasto mensal para os dados de teste   
 (X_test).
- As previsões são comparadas com os valores reais (y_test) usando as seguintes métricas:
  - **Erro Absoluto Médio (MAE)**:
    - Mede o erro médio em unidades absolutas, indicando o desvio médio entre os valores previstos e reais.
  - **Raiz do Erro Quadrático Médio (RMSE)**:
    - Penaliza erros maiores e fornece uma medida da variabilidade dos erros.

In [19]:
# Avaliar o modelo
y_pred = modelo.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# RESULTADO OBTIDO

In [20]:
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")

MAE: 47.0
RMSE: 62.69968101992226


Mesmo problema porem com outro modelo

# Gradient Boosting Regressor


1. Normalizar os dados:
- A normalização é essencial porque as variáveis independentes (X) podem ter escalas diferentes.
- O **StandardScaler** ajusta os dados para que tenham média 0 e desvio padrão 1.
- Isso ajuda modelos baseados em gradientes, como o Gradient Boosting, a convergir mais rápido.

In [24]:
# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

2. Dividir os dados em treino e teste:
- Separação dos dados em dois conjuntos:
  - Conjunto de treino (80%): Usado para treinar o modelo.
  - Conjunto de teste (20%): Usado para avaliar o modelo em dados desconhecidos.
- A divisão é feita de maneira aleatória, mas com um **random_state** fixo para reprodutibilidade.

In [25]:

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


3. Treinar o modelo com Gradient Boosting:
- O Gradient Boosting Regressor é um modelo baseado em boosting.
- Ele treina várias árvores de decisão em sequência, corrigindo os erros do modelo anterior.
- É uma escolha popular para problemas de regressão devido à sua robustez e capacidade de ajustar-se a padrões complexos.

In [26]:

# Treinar o modelo
modelo_gb = GradientBoostingRegressor(random_state=42)
modelo_gb.fit(X_train, y_train)


GradientBoostingRegressor(random_state=42)

4. Fazer previsões e avaliar o modelo:
- Após o treinamento, o modelo é testado em dados nunca vistos (X_test).
- As previsões são comparadas com os valores reais (y_test) usando métricas de avaliação:
  - **Erro Absoluto Médio (MAE)**:
    - Mede o erro médio absoluto entre as previsões e os valores reais.
    - Valores menores indicam melhor desempenho.
  - **Raiz do Erro Quadrático Médio (RMSE)**:
    - Penaliza erros maiores mais fortemente que o MAE.
    - Ideal para identificar a variabilidade dos erros do modelo.

In [27]:

# Avaliar
y_pred_gb = modelo_gb.predict(X_test)
mae_gb = mean_absolute_error(y_test, y_pred_gb)
rmse_gb = mean_squared_error(y_test, y_pred_gb, squared=False)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RESULTADO FINAL

In [28]:

print(f"MAE: {mae_gb}")
print(f"RMSE: {rmse_gb}")


MAE: 10.80580724056054
RMSE: 15.277493843764157


# Descrição do Problema

(Outro problema com outro modelo de treinamento)

O objetivo deste modelo é classificar se o gasto mensal de um endereço será alto ou baixo com base em características como o tipo de residência, tarifa de energia, economia e informações sobre energia solar. Essa classificação é útil para identificar padrões de consumo e sugerir estratégias personalizadas de economia para consumidores com alto consumo.


Carregamento e combinação de dados:
- Os arquivos JSON contêm informações sobre usuários, endereços e energia solar.
- A tabela "enderecos" será combinada com "energia_solar" usando a chave estrangeira "fk_endereco".
- Isso cria um dataset consolidado com as características relevantes para o modelo.


In [43]:
usuarios = carregar_json("usuarios.json")
enderecos = carregar_json("enderecos.json")
energia_solar = carregar_json("energia_solar.json")

In [35]:
# Merge dos dados
enderecos = enderecos.merge(energia_solar, left_on="id_endereco", right_on="fk_endereco", how="left")

Criação da variável-alvo:

O objetivo é classificar o gasto mensal como "alto" ou "baixo".
Para isso, calculamos a mediana do gasto mensal.
Entradas com gasto acima da mediana serão classificadas como "Alto" (1), e as demais como "Baixo" (0).
Isso transforma o problema de regressão em um problema de classificação binária.

In [32]:
# Criar variável alvo: Alto (1) ou Baixo (0) gasto
mediana = enderecos["gasto_mensal"].median()
enderecos["gasto_categoria"] = (enderecos["gasto_mensal"] > mediana).astype(int)

Seleção de variáveis relevantes:
- Para treinar o modelo, selecionamos as variáveis independentes que podem influenciar o gasto:
  - Tipo Residencial: Tipo de imóvel (e.g., Residencial, Comercial).
  - Tarifa: Custo por unidade de energia consumida.
  - Economia: Valor economizado com energia alternativa.
  - Área da Placa e Irradiação Solar: Dados de energia solar, relevantes para consumo autossuficiente.
- A variável dependente (y) será "gasto_categoria".

In [36]:
# Selecionar variáveis relevantes
dados = enderecos[[
    "tipo_residencial", "tarifa", "economia", "area_placa", "irradiacao_solar", "gasto_categoria"
]]
dados["tipo_residencial"] = LabelEncoder().fit_transform(dados["tipo_residencial"])

<ipython-input-36-296658bd41ed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados["tipo_residencial"] = LabelEncoder().fit_transform(dados["tipo_residencial"])


In [37]:
# Separar variáveis independentes (X) e dependente (y)
X = dados.drop(columns=["gasto_categoria"])
y = dados["gasto_categoria"]

- Dividimos os dados em duas partes:
  1. Treinamento (80% dos dados): Usados para ajustar o modelo.
  2. Teste (20% dos dados): Usados para avaliar o desempenho do modelo.
- Garantimos que os dados são embaralhados antes de dividir, para evitar viés.

In [39]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

Normalização das variáveis independentes:
- Como as variáveis possuem escalas diferentes (e.g., Tarifa em reais e Área da Placa em metros quadrados),
  aplicamos a normalização para garantir que todas contribuam igualmente para o modelo.

In [38]:
# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Treinamento do modelo:
- Utilizamos uma Árvore de Decisão para classificar os dados.
- Este modelo é simples, interpretável e eficiente para conjuntos de dados tabulares.
- Após o treinamento, avaliamos o modelo no conjunto de teste.

In [40]:
# Treinar o modelo
modelo = DecisionTreeClassifier(random_state=42)
modelo.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

Avaliação do desempenho:
- Métrica 1: Acurácia
  - Percentual de classificações corretas (i.e., predições certas / total de predições).
- Métrica 2: F1-Score
  - Média harmônica entre precisão e recall, útil para avaliar modelos com classes desbalanceadas.

In [41]:
# Avaliar o modelo
y_pred = modelo.predict(X_test)
acuracia = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [42]:
print(f"Acurácia: {acuracia * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")


Acurácia: 100.00%
F1-Score: 100.00%
